# Bloque 3 · Combinando recuperación y generación

En este bloque daremos el siguiente paso del flujo RAG: pasaremos de recuperar fragmentos (Bloque 2) a generar respuestas completas usando la API de RAG (`retrieve_and_generate`) de Amazon Bedrock. Analizaremos cómo el modelo genera texto a partir del contexto recuperado y practicaremos la personalización del *prompt*.

## ¿En qué se diferencia del Bloque 2?

- **Bloque 2**: nos enfocamos en recuperar fragmentos relevantes mediante el endpoint `Retrieve` y aprendimos a parsear la respuesta.
- **Bloque 3**: incorporamos un modelo generativo. Pedimos a Bedrock que, además de recuperar contextos, redacte una respuesta siguiendo nuestras instrucciones.

El LLM aporta redacción fluida, síntesis y la capacidad de adaptar el tono al público. Nuestro trabajo consiste en suministrarle el contexto correcto y un *prompt* claro para reducir al mínimo las alucinaciones.

## Rol del LLM dentro de un RAG

- **Interpreta la pregunta** y detecta la intención.
- **Lee los fragmentos recuperados** para extraer evidencia.
- **Redacta** una respuesta final, aplicando el tono y formato especificado.
- **Cita fuentes** si se lo pedimos explícitamente en el prompt.

Sin un buen prompt, el LLM puede ignorar información relevante o inventar detalles. Por eso es fundamental controlar el mensaje que le enviamos.

## API `retrieve_and_generate`

- Combina en una sola llamada la búsqueda en la knowledge base y la generación de texto.
- Permite indicar qué modelo usar (`modelArn`), cómo recuperar los fragmentos y cómo estructurar la generación (parámetros y *prompt template*).
- Devuelve tanto la respuesta generada como los fragmentos utilizados, para auditar y mostrar citas.

Trabajaremos directamente con esta API usando `boto3`.

## Diseño del prompt

Un *prompt template* típico para RAG incluye:
- Un mensaje al modelo sobre su rol (`Eres un asistente experto en ...`).
- Las instrucciones sobre cómo usar el contexto (`Solo responde usando la información proporcionada`).
- El espacio reservado para insertar los fragmentos (`{context}`) y la pregunta (`{query}`).
- Indicaciones de formato y tono.

La actividad práctica se centrará en modificar este template para observar cómo cambia la respuesta.

### Preparación antes de ejecutar

1. Ejecuta la celda de instalación de dependencias del **Bloque 0**.
2. Define las siguientes variables de entorno:
   - `BEDROCK_KB_ID`: identificador de la knowledge base.
   - `BEDROCK_MODEL_ARN`: ARN del modelo generativo disponible en tu cuenta (por ejemplo, Titan Text G1 o Claude 3 en Bedrock).
   - `BEDROCK_DATA_SOURCE_ID`: identificador del data source asociado a la knowledge base (necesario para lanzar ingestas).
   - `AWS_REGION`: solo si trabajas fuera de `us-east-1`.
3. Asegúrate de haber subido a S3 los documentos que quieras ingerir y de que el data source tenga acceso a esa ruta.

In [ ]:
import os
import time
from typing import Any, Dict

import boto3

AWS_REGION = os.environ.get("AWS_REGION", "us-east-1")
KNOWLEDGE_BASE_ID = os.environ.get("BEDROCK_KB_ID")
MODEL_ARN = os.environ.get("BEDROCK_MODEL_ARN")
DATA_SOURCE_ID = os.environ.get("BEDROCK_DATA_SOURCE_ID")

if not KNOWLEDGE_BASE_ID:
    raise ValueError("Falta la variable de entorno BEDROCK_KB_ID.")
if not MODEL_ARN:
    raise ValueError("Falta la variable de entorno BEDROCK_MODEL_ARN.")

rag_runtime = boto3.client("bedrock-agent-runtime", region_name=AWS_REGION)
rag_admin = boto3.client("bedrock-agent", region_name=AWS_REGION)

print("Clientes de Bedrock inicializados correctamente.")


## Actividad 1 · Ajustar el prompt y analizar la respuesta

1. Modifica el template para cambiar el tono, la estructura o los requisitos (por ejemplo, añadir una lista de pasos, citar la fuente exacta, responder en otro idioma, etc.).
2. Ejecuta la celda para generar una respuesta.
3. Observa cómo el modelo sigue (o no) tus instrucciones y revisa las citas devueltas.

In [ ]:
DEFAULT_PROMPT_TEMPLATE = """
Eres un asistente especializado en la plataforma interna de la empresa.
Responde a la pregunta usando únicamente la información de CONTEXTO.
Si la respuesta no está en el contexto, admite que no puedes responder.

CONTEXTO:
{context}

PREGUNTA:
{query}

Respuesta estructurada en párrafos cortos y menciona la fuente si está disponible.
""".strip()


def generar_con_prompt(
    pregunta: str,
    prompt_template: str,
    top_k: int = 4,
    max_tokens: int = 600,
    temperature: float = 0.2,
    top_p: float = 0.9,
) -> Dict[str, Any]:
    configuracion = {
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGE_BASE_ID,
            "modelArn": MODEL_ARN,
            "generationConfiguration": {
                "promptTemplate": {
                    "textPromptTemplate": prompt_template,
                },
                "temperature": temperature,
                "topP": top_p,
                "maxTokens": max_tokens,
            },
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults": top_k,
                }
            },
        },
    }

    respuesta = rag_runtime.retrieve_and_generate(
        input={"text": pregunta},
        retrieveAndGenerateConfiguration=configuracion,
    )
    return respuesta


In [ ]:
from textwrap import wrap


def mostrar_generacion(respuesta: Dict[str, Any]) -> None:
    output = respuesta.get("output", {})
    texto = output.get("text", "<Sin respuesta>")

    print("Respuesta generada:\n")
    for linea in wrap(texto, width=100):
        print(linea)

    print("\nCitas encontradas:")
    citas = respuesta.get("citations", [])
    if not citas:
        print("- No se devolvieron citas. Revisa el prompt o la coverage de la knowledge base.")
    else:
        for idx, cita in enumerate(citas, start=1):
            referencia = cita.get("reference", {})
            fuente = referencia.get("location", {}).get("s3Location", {}).get("uri", "Fuente desconocida")
            print(f"- Cita #{idx}: {fuente}")


pregunta_inicial = "¿Cuáles son las mejores prácticas para mantener actualizada la knowledge base de RAG?"
prompt_personalizado = DEFAULT_PROMPT_TEMPLATE

respuesta_generada = generar_con_prompt(pregunta_inicial, prompt_personalizado)
mostrar_generacion(respuesta_generada)


Experimenta cambiando `prompt_personalizado` y vuelve a ejecutar la celda anterior. Observa cómo el tono, la longitud y la presencia de citas dependen de tu template y de los parámetros de generación.

## Actividad 2 · Añadir nuevos documentos y repetir la pregunta

Objetivo: demostrar cómo un flujo RAG puede incorporar conocimiento reciente sin reentrenar el modelo.

1. Formula una pregunta sobre un documento **que todavía no esté** en la knowledge base.
2. Observa cómo responde el sistema (probablemente indicando que no tiene contexto suficiente).
3. Añade el documento al data source y lanza una nueva ingesta.
4. Vuelve a hacer la pregunta y verifica cómo ahora el LLM puede responder basándose en la nueva información.

In [ ]:
pregunta_nueva = "¿Qué pasos debemos seguir para desplegar el chatbot en el entorno de pruebas?"
prompt_para_nuevo_doc = DEFAULT_PROMPT_TEMPLATE

respuesta_sin_doc = generar_con_prompt(pregunta_nueva, prompt_para_nuevo_doc)
mostrar_generacion(respuesta_sin_doc)


Si la knowledge base todavía no contiene el documento, la respuesta debería dejar en claro que no existe suficiente contexto. Ahora lanzaremos una ingesta para incorporar la nueva información.

### Lanzar una nueva ingesta desde código

- Sube el documento (por ejemplo, `procedimiento_chatbot.pdf`) al bucket configurado en tu data source.
- Confirma que `BEDROCK_DATA_SOURCE_ID` tiene el identificador correcto.
- Ejecuta la siguiente celda para iniciar el job de ingesta y espera a que finalice.

In [ ]:
if not DATA_SOURCE_ID:
    raise ValueError("Define la variable BEDROCK_DATA_SOURCE_ID para lanzar ingestas desde código.")

ingestion_job = rag_admin.start_ingestion_job(
    knowledgeBaseId=KNOWLEDGE_BASE_ID,
    dataSourceId=DATA_SOURCE_ID,
)

job_id = ingestion_job["ingestionJob"]["ingestionJobId"]
print(f"Job de ingesta iniciado: {job_id}")

estado_final = None
while True:
    detalle = rag_admin.get_ingestion_job(
        knowledgeBaseId=KNOWLEDGE_BASE_ID,
        dataSourceId=DATA_SOURCE_ID,
        ingestionJobId=job_id,
    )
    estado = detalle["ingestionJob"]["status"]
    print(f"Estado actual: {estado}")
    if estado in {"COMPLETE", "FAILED", "STOPPED"}:
        estado_final = estado
        break
    time.sleep(15)

if estado_final != "COMPLETE":
    raise RuntimeError(f"La ingesta no finalizó correctamente (estado: {estado_final}).")

print("La knowledge base se actualizó con la nueva información.")


Una vez completada la ingesta, vuelve a ejecutar la consulta con la misma pregunta para comprobar la diferencia.

In [ ]:
respuesta_con_doc = generar_con_prompt(pregunta_nueva, prompt_para_nuevo_doc)
mostrar_generacion(respuesta_con_doc)


## Reflexión final

- Un *prompt* bien redactado guía al modelo para usar únicamente la evidencia recuperada.
- La API de RAG permite iterar rápidamente: añadir documentos y volver a preguntar sin reentrenar el modelo.
- La trazabilidad (citas) es clave para generar confianza en el chatbot.

En el siguiente bloque integraremos todo esto dentro de Chainlit para ofrecer una experiencia conversacional completa.